# Combining the Kar et al. and NIST databases

Shambles of code

In [ ]:
# import the libraries
import json

# import the databases
nist = json.load(open('2021-03-29_APF_NIST consolidated_01.json'))
wckb = json.load(open('2021-03-25_APF_WCKB reactions + references.json'))


# loop through the enzymes in both databases
for enzyme, value in nist.items():
    # lists of the database varialbes
    keq_values_per_enzyme = []
    km_values_per_enzyme = []
    enthalpy_values_per_enzyme = []
    temperatures_per_enzyme = []
    phs_per_enzyme = []
    
    # lists of identifying whether the reference contains the identified variable 
    references_of_an_enzyme = []
    reaction_of_an_enzyme = []
    km_in_the_reference = []
    enthalpy_in_the_reference = []
    keqs_in_a_reference = []
    for enzyme2, value2 in wckb.items():
        if enzyme == enzyme2:
            
            
            
            
    #store the information into a nested dictionary structure
    data_per_enzyme[enzyme] = {'reaction':reaction_of_an_enzyme,
                               'experimental temperatures':temperatures_per_enzyme,
                               'experimental phs':phs_per_enzyme,
                               'keq reference':references_of_an_enzyme,
                               'Keq':{'keq values in the reference':keqs_in_a_reference,
                                       'keqs':keq_values_per_enzyme, 
                                       'keq quantity':len(keq_values_per_enzyme), 
                                       'keq average':average_keq_per_enzyme, 
                                       'keq standard deviation':standard_deviation_keq_per_enzyme},
                               'Km':{'km values in the reference':km_in_the_reference,
                                    'km values':km_values_per_enzyme,
                                    'km average':average_km_per_enzyme,
                                    'km standard deviation':standard_deviation_km_per_enzyme},
                               'Enthalpy':{'enthalpy values in the reference':enthalpy_in_the_reference,
                                         'enthalpy values':enthalpy_values_per_enzyme,
                                         'enthalpy average':average_enthalpy_per_enzyme,
                                         'enthalpy standard deviation':standard_deviation_enthalpy_per_enzyme}}

## Comparing the data with the BiGG model

The code is illogical in a few parts, and is inefficient -- The aforementioned reactions data is compared with the BiGG model for the core E. coli metabolism. The missing enzymes will be identified elsewhere to provide a working dataset for our simulation model.

In [ ]:
#import the libraries and JSON files
import collections
import json
import re

metabolites = json.load(open('2021-03-17_APF_Karr et al. metabolite data.json'))
reactions = json.load(open('2021-03-17_APF_Karr et al. kinetic data.json'))
nist = json.load(open('2021-03-21_APF_NIST consolidated_01.json'))
bigg = json.load(open('BiGG model of E_coli core metabolism_indented.json'))


bigg_model_ecoli_ids = ['ACALD', 'ACALDt', 'ACKr','ACONTa','ACONTb','ACt2r','ADK1','AKGDH','AKGt2r','ALCD2x',
                        'ATPM','ATPS4r','BIOMASS_Ecoli_core_w_GAM','CO2t','CS','CYTBD','D_LACt2','ENO','ETOHt2r',
                        'EX_acald_e','EX_ac_e','EX_akg_e','EX_co2_e','EX_etoh_e','EX_for_e','EX_fru_e',
                        'EX_fum_e','EX_glc__D_e','EX_gln__L_e','EX_glu__L_e','EX_h2o_e','EX_h_e','EX_lac__D_e',
                        'EX_mal__L_e','EX_nh4_e','EX_o2_e','EX_pi_e','EX_pyr_e','EX_succ_e','FBA','FBP',
                        'FORt','FORt2','FRD7','FRUpts2','FUM','FUMt2_2','G6PDH2r','GAPD','GLCpts',
                        'GLNabc','GLNS','GLUDy','GLUN','GLUSy','GLUt2r','GND','H2Ot','ICDHyr','ICL',
                        'LDH_D','MALS','MALt2_2','MDH','ME1','ME2','NADH16','NADTRHD','NH4t','O2t','PDH',
                        'PFK','PFL','PGI','PGK','PGL','PGM','PIt2r','PPC','PPCK','PPS','PTAr','PYK','PYRt2','RPE',
                        'RPI','SUCCt2_2','SUCCt3','SUCDi','SUCOAS','TALA','THD2','TKT1','TKT2','TPI']



bigg_model_ecoli_names = ['Acetaldehyde dehydrogenase (acetylating)', 'Acetaldehyde reversible transport',
                         'Acetate kinase','Aconitase (half-reaction A, Citrate hydro-lyase)',
                         'Aconitase (half-reaction B, Isocitrate hydro-lyase)',
                         'Acetate reversible transport via proton symport', 'Adenylate kinase',
                         '2-Oxogluterate dehydrogenase', '2 oxoglutarate reversible transport via symport',
                         'Alcohol dehydrogenase (ethanol)', 'ATP maintenance requirement', 
                         'ATP synthase (four protons for one ATP)', 'Biomass Objective Function with GAM',
                         'CO2 transporter via diffusion', 'Citrate synthase', 
                         'Cytochrome oxidase bd (ubiquinol-8: 2 protons)','D lactate transport via proton symport',
                         'Enolase','Ethanol reversible transport via proton symport','Acetaldehyde exchange',
                         'Acetate exchange', '2-Oxoglutarate exchange', 'CO2 exchange', 'Ethanol exchange',
                         'Formate exchange', 'D-Fructose exchange', 'Fumarate exchange', 'D-Glucose exchange',
                         'L-Glutamine exchange', 'L-Glutamate exchange', 'H2O exchange','H+ exchange','D-lactate exchange',
                         'L-Malate exchange','Ammonia exchange','O2 exchange','Phosphate exchange','Pyruvate exchange',
                         'Succinate exchange','Fructose-bisphosphate aldolase','Fructose-bisphosphatase',
                         'Formate transport via diffusion','Formate transport in via proton symport','Fumarate reductase',
                         'Fructose transport via PEP:Pyr PTS (f6p generating)','Fumarase',
                         'Fumarate transport via proton symport (2 H)','Glucose 6-phosphate dehydrogenase',
                         'Glyceraldehyde-3-phosphate dehydrogenase','D-glucose transport via PEP:Pyr PTS',
                         'L-glutamine transport via ABC system','Glutamine synthetase','Glutamate dehydrogenase (NADP)',
                         'Glutaminase','Glutamate synthase (NADPH)','L glutamate transport via proton symport reversible',
                         'Phosphogluconate dehydrogenase', 'H2O transport via diffusion','Isocitrate dehydrogenase (NADP)',
                         'Isocitrate lyase','D-lactate dehydrogenase','Malate synthase',
                         'Malate transport via proton symport (2 H)','Malate dehydrogenase','Malic enzyme (NAD)',
                         'Malic enzyme (NADP)','NADH dehydrogenase (ubiquinone-8 & 3 protons)',
                         'NAD transhydrogenase','Ammonia reversible transport','O2 transport diffusion',
                         'Pyruvate dehydrogenase','Phosphofructokinase','Pyruvate formate lyase',
                         'Glucose-6-phosphate isomerase','Phosphoglycerate kinase','6-phosphogluconolactonase',
                         'Phosphoglycerate mutase','Phosphate reversible transport via symport',
                         'Phosphoenolpyruvate carboxylase','Phosphoenolpyruvate carboxykinase',
                         'Phosphoenolpyruvate synthase','Phosphotransacetylase','Pyruvate kinase',
                         'Pyruvate transport in via proton symport','Ribulose 5-phosphate 3-epimerase',
                         'Ribose-5-phosphate isomerase','Succinate transport via proton symport (2 H)',
                         'Succinate transport out via proton antiport','Succinate dehydrogenase (irreversible)',
                         'Succinyl-CoA synthetase (ADP-forming)','Transaldolase','NAD(P) transhydrogenase',
                         'Transketolase','Transketolase','Triose-phosphate isomerase']


#creating a complete list of enzymatic reaction names in the BiGG model
enzyme_names = []
metabolite_names = []
for keys in bigg:
    for key, values in  keys.items():
        if key == 'reactions':
            for key, value in values.items():
                enzyme_names.append(value['name'])
                
                
                #creating a list of metabolite names in the BiGG model
                for entry in value['metabolites']:
                    for key, val in entry.items():
                        if key == 'bigg_id':
                            if val not in metabolite_names:
                                metabolite_names.append(val)


#establish the complete set of reaction names
karr_reaction_names = []
nist_enzyme_name = []
for id, values in reactions.items():
    karr_reaction_names.append(values['Enzyme Name'])
for id, value in nist.items():
    nist_enzyme_name.append(id)

                                
#combine the NIST and Karr datasets while respecting the case sensitivity of the enzyme names
union_reactions = []
intersection_reactions = []
for enzyme in karr_reaction_names:
    for enzyme2 in nist_enzyme_name:
        if enzyme.lower() == enzyme2.lower(): 
            if enzyme not in intersection_reactions:
                intersection_reactions.append(enzyme)
        if enzyme not in union_reactions:
            union_reactions.append(enzyme)
        if enzyme2 not in union_reactions:
            union_reactions.append(enzyme2)
            
            
#determine the set of BiGG reactions that are described or undescribed by the set of scraped values 
described_from_union = []
described_from_intersection = []
described_in_karr = []
described_in_karr_by_karr = []
for enzyme in enzyme_names:
    for enzyme2 in union_reactions:
        if enzyme.lower() == enzyme2.lower() and enzyme not in described_from_union:
            described_from_union.append(enzyme)
            break
    for enzyme3 in intersection_reactions:
        if enzyme.lower() == enzyme3.lower() and enzyme not in described_from_intersection:
            described_from_intersection.append(enzyme)
            break
    for enzyme4 in karr_reaction_names:
        if enzyme.lower() == enzyme4.lower() and enzyme not in described_in_karr:
            described_in_karr.append(enzyme)
            if enzyme4 not in described_in_karr_by_karr:
                described_in_karr_by_karr.append(enzyme4)
            break
            
            
# determining the set fo reactions that are not described by the scraped data
undescribed_from_union = []
undescribed_from_intersection = []          
for enzyme in enzyme_names:
    if enzyme not in described_from_union:
        undescribed_from_union.append(enzyme)
for enzyme in enzyme_names:    
    if enzyme not in described_from_intersection:
        undescribed_from_intersection.append(enzyme)
        
both_enzymes_intersection = []
both_enzymes_union = []
for enzyme in undescribed_from_intersection:
    if enzyme in described_from_intersection:
        both_enzymes_intersection.append(enzyme)
for enzyme in undescribed_from_union:
    if enzyme in described_from_union:
        both_enzymes_union.append(enzyme)
    
    
'''#create the michaelis-menten equations for each of the enzymes
described_kinetic_information = {}
for id, values in reactions.items():
    #print(values['Enzyme Name'])
    if values['Enzyme Name'] in described_in_karr or values['Enzyme Name'] in described_in_karr_by_karr:
        print(reactions[id])'''
    
    
'''
#identify the complete set of metabolite names 
karr_metabolite_names = []
for id, list in metabolites.items():
    karr_metabolite_names.append(list['Name'])   

    
#identify the complete set of metabolite ids
metabolite_ids = []
for key in metabolites:
    metabolite_ids.append(key) '''

    

#identifying the differences between the captured enzyme list and the original BiGG list
combined_union = described_from_union + undescribed_from_union
combined_intersection = described_from_intersection + undescribed_from_intersection
for enzyme in combined_intersection:
    if enzyme not in enzyme_names:
        print(enzyme)
for enzyme in combined_union:
    if enzyme not in enzyme_names:
        print(enzyme)
intersection_difference = collections.Counter(enzyme_names) - collections.Counter(combined_intersection)
union_difference = collections.Counter(enzyme_names) - collections.Counter(combined_union)
print('intersection difference\n',intersection_difference)
print('union difference\n',union_difference)
    
    

print('\nIntersection reactions', len(intersection_reactions))
print('Union reactions', len(union_reactions))

print('\nTotal BiGG enzymes:', len(enzyme_names))
print('Described from union enzymes:', len(described_from_union))
print('Undescribed from union reactions', len(undescribed_from_union))
print('Described from intersection enzymes', len(described_from_intersection))
print('Undescribed from intersection enzymes', len(undescribed_from_intersection))

print('\nDescribed in Karr: %s\n' %(len(described_in_karr)), described_in_karr)


union_reactions.sort()
#print(union_reactions)
intersection_reactions.sort()
print('\nInersection reactions\n', '%s'%('='*len('Inersection reactions')),'\n',intersection_reactions)
print('\nBiGG & intersection reactions\n', '%s'%('='*len('BiGG & intersection reactions')), '\n',described_from_intersection)


'''print('\nDescribed and undescribed intersection reactions\n', 
       '%s'%('='*len('nDescribed and undescribed intersection reactions')), 
       '\n%s' %(both_enzymes_intersection))
print('\nDescribed and undescribed union reactions\n', 
       '%s'%('='*len('nDescribed and undescribed union reactions')), 
       '\n%s' %(both_enzymes_union))'''


'''keys.sort()
for key in keys:
    print(key)'''


The representation of the BiGG model from the NIST database is determined. The set of reactions that are represented and unrepresented are furthermore separately stored as lists.

In [46]:
#import the libraries and JSON files
import collections
import numpy
import json
import re

nist = json.load(open('2021-03-21_APF_NIST consolidated_01.json'))
bigg = json.load(open('BiGG model of S. aureus.json'))
empty_cell = ['nan', None, numpy.nan, '', ' ']

# creating a complete list of enzymatic reaction names in the BiGG model
saureus_enzymes = []
for key, value in bigg.items():
    #print(key)
    if key == 'reactions':
        for enzyme in value:
            for key2, value2 in enzyme.items():
                if key2 == 'name':
                    saureus_enzymes.append(value2)

# compiling the set of reaction names from the NIST database
nist_enzyme_name = []
parenthesized_nist_enzymes = []
for id, value in nist.items():
    #print(id)
    nist_enzyme_name.append(id)
    if re.search('(\s\(.*\))', id):
        striped_value = re.sub('(\s\(.*\))', '', id)
        #print(striped_value)
        parenthesized_nist_enzymes.append(striped_value)

#print(parenthesized_nist_enzymes)
        
# identifying the BiGG model enzymes that are represented by the NIST database 
represented_enzymes = []
represented_nist_enzymes = []
for enzyme in saureus_enzymes:
    for enzyme2 in nist_enzyme_name:
        if enzyme.lower() == enzyme2.lower(): 
            if enzyme not in represented_enzymes:
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(enzyme2)
    for enzyme3 in parenthesized_nist_enzymes:
        if re.search(enzyme3, enzyme):
            #print(enzyme)
            if enzyme not in represented_enzymes:
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(enzyme2)
                
unexpressed_enzymes = (set(nist_enzyme_name) - set(represented_nist_enzymes))
unexpressed_nist_enzymes_counted = len(nist_enzyme_name) - len(represented_enzymes)
    
print('\nTotal BiGG enzymes:', len(saureus_enzymes))
print('\nTotal unexpressed NIST enzymes:', unexpressed_nist_enzymes_counted)
print('\nUnexpressed enzymes from the NIST database: %s\n' %(len(unexpressed_enzymes)), unexpressed_enzymes)

print('\nRepresented by the NIST database: %s\n' %(len(represented_enzymes)), represented_enzymes)



Total BiGG enzymes: 1455

Total unexpressed NIST enzymes: 343

Unexpressed enzymes from the NIST database: 345
 {'_-alanyl-CoA ammonia lyase', 'leucyl aminopeptidase', 'alanine aminotransferase', 'papain', 'pyrimidine-nucleoside phosphorylase', 'acetate CoA-transferase', '2,3-dimethylmalate lyase', 'myosin ATPase', 'L-rhamnose isomerase', '2-aminoadipate transaminase', 'glutamine-pyruvate aminotransaminase', 'uridine phosphorylase', 'S)-2-methylmalate dehydratase', 'glucose-6-phosphate 1-epimerase', 'acetylcholinesterase', 'thiamine pyridinylase', 'aryl sulfotransferase', 'N-methyl-2-oxoglutaramate hydrolase', 'cyclomaltodextrin glucanotransferase and a-amylase', 'a-glucosidase', 'lysine 2,3-aminomutase', 'quinate dehydrogenase', 'malate dehydrogenase', 'glycine formiminotransferase', '2-dehydro-3-deoxyphosphogluconate aldolase', '3-oxoacyl-[acyl-carrier-protein] reductase', 'formate dehydrogenase (NADP+)', 'allantoicase', '2-dehydro-3-deoxy-6-phosphogalactonate aldolase', 'D-xylulose

The thermodynamically described enzymes are parsed from the NIST database data. The enzymes which are conjoined with "and" were separated and individually analyzed. The sets of separately described enzymes and conjoined enzymes are each applied as columns of a dataframe, which is exported.  

In [198]:
#import the libraries and JSON files
from ftfy import fix_encoding
import pandas
import numpy
import json
import re

nist = json.load(open('2021-03-29_APF_NIST consolidated_01.json'))
bigg = json.load(open('BiGG model of S. aureus.json'))
empty_cell = ['nan', None, numpy.nan, '', ' ', 'NaN', '[]']


# creating a complete list of enzymatic reaction names in the BiGG model
saureus_enzymes = []
for key, value in bigg.items():
    #print(key)
    if key == 'reactions':
        for enzyme in value:
            for key2, value2 in enzyme.items():
                if key2 == 'name':
                    saureus_enzymes.append(value2)
                    

# compiling the set of reaction names from the NIST database
nist_enzyme_name = []
parenthesized_nist_enzymes = []
parenthesized_nist_ennzymes_original = []
and_conjoined_enzymes = []
and_conjoined_enzyme_original = []
thermodynamic_enzyme_names = []
kinetic_enzymes = []
for name, value in nist.items():
    nist_enzyme_name.append(name)
    for detail, information in value.items():
        #print(detail, '\t', information)
        if detail == 'Keq':
            for detail2, information2 in information.items():
                if detail2 == 'keqs':
                    for information3 in information2:
                        #print(id, '\t', information3)
                        if information3 not in empty_cell: 
                            #print('keqs', id)
                            if re.search('(\sand\s)', name):
                                left_enzyme_striped_value = re.sub('(.*and\s)', '', name)
                                right_enzyme_striped_value = re.sub('(\sand.*)', '', name)
                                if left_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(right_enzyme_striped_value)
                                if left_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(right_enzyme_striped_value)
                                if name not in and_conjoined_enzyme_original:
                                    and_conjoined_enzyme_original.append(name)
                            else:
                                if name not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(name)
                        else:
                            print(information3)

        if detail == 'Enthalpy':
            for detail2, information2 in information.items():
                #print('yay')
                #print('information:\t', detail2, '\t', information2)
                if detail2 == 'enthalpy values':
                    #print('yes')
                    for information3 in information2:
                        if information3 not in empty_cell:
                            if re.search('(\sand\s)', name):
                                left_enzyme_striped_value = re.sub('(.*and\s)', '', name)
                                right_enzyme_striped_value = re.sub('(\sand.*)', '', name)
                                if left_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(right_enzyme_striped_value)    
                                if left_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(right_enzyme_striped_value)
                                if name not in and_conjoined_enzyme_original:
                                    and_conjoined_enzyme_original.append(name)
                            else:
                                if name not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(name)
                        else:
                             print(information3)   
     
        if detail == 'Km':
            for detail2, information2 in information.items():
                #print('yay')
                #print('information:\t', detail2, '\t', information2)
                if detail2 == 'km values':
                    #print('yes')
                    for information3 in information2:
                        if information3 not in empty_cell:
                            if name not in kinetic_enzymes:
                                kinetic_enzymes.append(name)
                                #print('kinetics', name)
                        else:
                             print(information3)           
                                
        
        if re.search('(\s\(.*\))', name):
            striped_value = re.sub('(\s\(.*\))', '', name)
            #print(striped_value)
            parenthesized_nist_enzymes.append(striped_value)
            parenthesized_nist_ennzymes_original.append(name)
            
                
'''for enzyme in and_conjoined_enzyme_original:
    index = and_conjoined_enzyme_original.index(enzyme)
    for entry in nist[enzyme]['Keq']['keqs']:
        if entry not in empty_cell and and_conjoined_enzymes[index] not in thermodynamic_enzyme_names:
            thermodynamic_enzyme_names.append(enzyme)
    for entry in nist[enzyme]['Enthalpy']['enthalpy values']:
        if entry not in empty_cell and and_conjoined_enzymes[index] not in thermodynamic_enzyme_names:
            thermodynamic_enzyme_names.append(enzyme)'''
            
                
'''print('separated enzymes: %s\n' %(len(and_conjoined_enzymes)), set(and_conjoined_enzymes))   
print('\noriginal enzymes: %s\n' %(len(and_conjoined_enzyme_original)), set(and_conjoined_enzyme_original))     '''       

'''print(parenthesized_nist_enzymes)
nist_enzyme_name += and_conjoined_enzymes
thermodynamic_enzyme_names.append('test')'''
        
    
# comparing the NIST and BiGG databases 
represented_enzymes = []
represented_nist_enzymes = []
for enzyme in saureus_enzymes:
    for enzyme2 in thermodynamic_enzyme_names:
        if enzyme.lower() == enzyme2.lower():
            represented_enzymes.append(enzyme)
            represented_nist_enzymes.append(enzyme2)
    
    
'''    for enzyme2 in nist_enzyme_name:
        if enzyme.lower() == enzyme2.lower(): 
            if enzyme not in represented_enzymes:
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(enzyme2)
    for enzyme3 in parenthesized_nist_enzymes:
        if re.search(enzyme3, enzyme):
            #print(enzyme)
            if enzyme not in represented_enzymes:
                index = parenthesized_nist_enzymes.index(enzyme3)
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(parenthesized_nist_ennzymes_original[index])
    for enzyme3 in and_conjoined_enzymes:
        if re.search(enzyme3, enzyme):
            #print(enzyme)
            if enzyme not in represented_enzymes:
                print(enzyme, '\t', enzyme3)
                index = and_conjoined_enzymes.index(enzyme3)
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(and_conjoined_enzyme_original[index])'''
                
                
'''unexpressed_enzymes = (set(nist_enzyme_name) - set(represented_nist_enzymes))
unexpressed_nist_enzymes_counted = len(nist_enzyme_name) - len(represented_enzymes)
not_thermodynamic_enzymes = set(nist_enzyme_name) - set(thermodynamic_enzyme_names)
    
print('\nTotal BiGG enzymes:', len(saureus_enzymes))
print('\nNot thermodynamic enzymes:', len(not_thermodynamic_enzymes))
print('\nTotal unexpressed NIST enzymes:', unexpressed_nist_enzymes_counted)
print('\nUnexpressed enzymes from the NIST database: %s\n' %(len(unexpressed_enzymes)), unexpressed_enzymes)
print('\nThermodynamic enzymes in the NIST database: %s\n' %(len(thermodynamic_enzyme_names)), 
      thermodynamic_enzyme_names)
print('\nS. aureus represented by the NIST database: %s\n' %(len(represented_enzymes)), represented_enzymes)
print('\nKinetics in the NIST database: %s\n' %(len(kinetic_enzymes)), kinetic_enzymes)'''

'''thermodynamic_dictionary = {}
for enzyme in thermodynamic_enzyme_names:
    thermodynamic_dictionary[enzyme] = '''


# export the parsed reaction names to a CSV file
'''dataframe = pandas.DataFrame.from_dict({'separated enzymes':thermodynamic_enzyme_names,
                                     'conjoined enzymes':and_conjoined_enzyme_original},
                            orient = 'index')
dataframe = dataframe.transpose()
dataframe.to_csv('2021-04-29_APF_thermodynamically described enzymes_01.csv')'''



# determine the set of metabolites in the thermodynamically described reactions
'''described_compounds = []
for enzyme in nist:
    for enzyme2 in thermodynamic_enzyme_names:
        if re.search('(-)', enzyme2):
            enzyme2 = re.sub('(-)', '(\-)', enzyme2)
        if re.search(enzyme2, enzyme):
            #print(enzyme,'\n', enzyme2)
            reactants_list = []
            products_list = []
            for reaction in nist[enzyme]['reaction']:
                #print(reaction)
                reaction = fix_encoding(reaction)
                #print(reaction)
                if re.search('(=)', reaction):
                    reaction = reaction.split('=')
                    #print('reaction:\t', reaction)
                    reactants = reaction[0]
                    products = reaction[1] 
                    #print('reactants:\t', reactants)
                    #print('products:\t', products)
                    reactants_list = reactants.split('+')
                    products_list = products.split('+')
                    #print('reactants list:\t', reactants_list)
                    #print('products list:\t', products_list)
                    for element in reactants_list:
                        #print(element)
                        element = element.strip()
                        element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                        element = re.sub('(\d\s|\d\/\d\s)', '', element)
                        if element not in described_compounds:
                            #print(element)
                            described_compounds.append(element)
                    for element in products_list:
                        #print(element)
                        element = element.strip()
                        element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                        element = re.sub('(\d\s|\d\/\d\s)', '', element)
                        if element not in described_compounds:
                            #print(element)
                            described_compounds.append(element)
                else:
                    print('abnormal reaction:\t', reaction)

export_comopunds_yes_no = {}
for compound in described_compounds:
    export_comopunds_yes_no[compound] = {'logical name': 'yes',
                                        'present in KEGG': ''}

with open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_02.json', 'w') as output:
    json.dump(export_comopunds_yes_no, output, indent = 4)'''
    

# create the clean set of reactions for the ModelSEED repository code
rearranged_reactions_for_modelseed = []
to_be_rearranged = []
export_reactions_for_modelseed = {}
for enzyme in nist:
    for enzyme2 in thermodynamic_enzyme_names:
        if re.search('(-)', enzyme2):
            enzyme2 = re.sub('(\-)', '\-', enzyme2)
        if re.search(enzyme2, enzyme):
            for reaction in nist[enzyme]['reaction']:
                reaction = fix_encoding(reaction)
                #print(reaction)
                if reaction not in to_be_rearranged:
                    to_be_rearranged.append(reaction)
                    #print(reaction)
                    
                    # replace the phase identifiers
                    if re.search('(\(aq\)|\(l\)|\(sln\)|\(liq\))', reaction):
                        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
                    #print(reaction)
                    
                    # arrange the reaction equilibrium arrow and stoichiometric coefficients for ModelSEED
                    if re.search('(\s?=\s?)', reaction):
                        reaction = re.sub('(\s?=\s?)', ' <=> ', reaction)
                    if re.search('((?<!\w)\d\s|\d\/\d\s)', reaction):
                        reaction = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'(\1) ', reaction)
                        reaction = re.sub('(?<=\d)(\s)(?=\))', '', reaction)
                    else:
                        reaction = re.sub('(?<=\+)(\s)(?=\w)', ' (1) ', reaction)
                        reaction = re.sub('(?<=>)(\s)(?=\w)', ' (1) ', reaction)
                    if reaction[0] == ' ':
                        reaction = re.sub('\s', '(1) ', reaction, 1)
                    else:
                        reaction = '(1) ' + reaction
                    reaction = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction)
                    reaction += '[0]'
                    #print(reaction)
                    if reaction not in rearranged_reactions_for_modelseed:
                        enzyme2 = re.sub(r'(\\)', '', enzyme2)
                        export_reactions_for_modelseed[enzyme2] = {'reaction expression':reaction,
                                                                   'yes_no':''}
                        rearranged_reactions_for_modelseed.append(reaction)
                    else:
                        print('repeated reactions:\t', reaction)


'''print('\nFinal reactions:')
for reaction in rearranged_reactions_for_modelseed:
    print(reaction)
'''
    
with open('2021-04-29_APF_nist thermodynamically described reactions for ModelSEED_02.json', 'w') as output:
    json.dump(export_reactions_for_modelseed, output, indent = 4)


repeated reactions:	 (1) α-D-glucose 1-phosphate[0] <=> (1) α-D-glucose 6-phosphate[0]
repeated reactions:	 (1) α-D-galactose 1-phosphate[0] <=> (1) α-D-glucose 1-phosphate[0]
repeated reactions:	 (1) sedoheptulose 7-phosphate[0] + (1) D-glyceraldehyde 3-phosphate[0] <=> (1) D-erythrose 4-phosphate[0] + (1) D-fructose 6-phosphate[0]
repeated reactions:	 (1) diolein[0] + (1) H2O[0] <=> (1) oleic acid[0] + (1) monoolein[0]
repeated reactions:	 (1) monoolein[0] + (1) H2O[0] <=> (1) oleic acid[0] + (1) glycerol[0]
repeated reactions:	 (1) triolein[0] + (1) H2O[0] <=> (1) oleic acid[0] + (1) diolein[0]
repeated reactions:	 (1) N-acetyl-L-phenylalanine ethyl ester[0] + (1) H2O[0] <=>  N-acetyl-L-phenylalanine[0] + (1) ethanol[0]
repeated reactions:	 (1) 3,4,5-trihydroxybenzoic acid propyl ester[0] + (1) H2O[0] <=> (1) 3,4,5-trihydroxybenzoate[0] + (1) propan-1-ol[0]


In [89]:
import re 
from ftfy import fix_encoding

j = 'ATP-\u00ce\u00b1-S(aq) + (R)-5-phosphomevalonate(aq) = ADP-\u00ce\u00b1-S(aq) + (R)-5-diphosphomevalonate(aq)'
p = fix_encoding(j)
print(p)



def decodeu (match):
    u = '%c%c' % (int(match.group(1), 16), int(match.group(2), 16))
    return repr(u.decode('utf-8'))[2:8]




j = re.sub(r'\\u00([cd][0-9a-f])\\u00([89ab][0-9a-f])',decodeu, j)

print(j)

ATP-α-S(aq) + (R)-5-phosphomevalonate(aq) = ADP-α-S(aq) + (R)-5-diphosphomevalonate(aq)
ATP-Î±-S(aq) + (R)-5-phosphomevalonate(aq) = ADP-Î±-S(aq) + (R)-5-diphosphomevalonate(aq)


In [197]:
import json
import re

wckb = json.load(open('2021-03-25_APF_WCKB reactions + references.json'))
wckb_metabolites = json.load(open('2021-03-17_APF_Karr et al. metabolite data.json'))

empty_cells = ['NaN', None, numpy.nan, '', ' ', 'null', 'None', 'nan', '[None]', '[\'None\']']    

# determine the set of thermodynamically described reactions from the WCKB downloaded json data
thermodynamically_described_enzymes_wckb = []
wckb_enzymes = []
wckb_keqs = []
wckb_kinetics = []
wckb_gibbs = []
for enzyme, value in wckb.items():
    if enzyme not in wckb_enzymes:
        #print(enzyme)
        wckb_enzymes.append(enzyme)
        for keq in value['keq']:
            if keq not in empty_cells and enzyme not in wckb_gibbs:
                thermodynamically_described_enzymes_wckb.append(enzyme)
                wckb_keqs.append(enzyme)
        for gibbs in value['Gibbs free energy']:
            if gibbs not in empty_cells and enzyme not in wckb_gibbs:
                thermodynamically_described_enzymes_wckb.append(enzyme)
                wckb_gibbs.append(enzyme)
        for forward_kinetics in value['forward kinetics']:
            if forward_kinetics not in empty_cells and enzyme not in wckb_gibbs:
                wckb_kinetics.append(enzyme)
        for backward_kinetics in value['backward kinetics']:
            if backward_kinetics not in empty_cells and enzyme not in wckb_gibbs:
                wckb_kinetics.append(enzyme)
                
print('\n\nWCKB enzymes:\t\t', len(wckb_enzymes)) 
print('Thermodynamically described enzymes: ', len(thermodynamically_described_enzymes)) 
print('WCKB Keqs:\t\t', len(wckb_keqs))   
print('WCKB Kinetics:\t\t', len(wckb_kinetics))
print('WCKB gibbs:\t\t', len(wckb_gibbs))     
                            
thermodynamically_described_enzymes = set(wckb_gibbs).union(set(wckb_keqs))
wckb_reactions = []
export_reactions_wckb_for_modelseed = {}
for enzyme in thermodynamically_described_enzymes:
    for entry in wckb[enzyme]['stoichiometry']:
        compound_loop = 0
        product_loop = 0
        for molecule in entry:
            if molecule['compartment'] == 'c':
                compartment_value = 0
            elif molecule['compartment'] == 'e':
                compartment_value = 1
            elif molecule['compartment'] == 'm':
                compartment_value = 2
                
            if compound_loop == 0:
                reaction_string = '(%s) %s[%d]'  %(abs(int(float(molecule['coefficient']))), molecule['molecule'], compartment_value)
                compound_loop += 1
            elif product_loop == 0 and float(molecule['coefficient']) > 0: 
                reaction_string += ' <=> (%s) %s[%d]'  %(int(float(molecule['coefficient'])), molecule['molecule'], compartment_value)            
                product_loop += 1
            else:
                reaction_string += ' + (%s) %s[%d]'  %(abs(int(float(molecule['coefficient']))), molecule['molecule'], compartment_value)
        print(reaction_string)
        wckb_reactions.append(reaction_string)       
        export_reactions_wckb_for_modelseed[enzyme] = {'reaction expression':reaction_string,
                                                       'yes_no':''}
                    
          
                
with open('2021-04-29_APF_wckb thermodynamically described reactions for ModelSEED_01.json', 'w') as output:
    json.dump(export_reactions_wckb_for_modelseed, output, indent = 4)
    
    
# determine the set of thermodynamically described compounds from the aforementioned reactions
'''described_compounds = []
for enzyme in thermodynamically_described_enzymes:
    for entry in wckb[enzyme]['stoichiometry']:
        for molecule in entry:
            if molecule['molecule'] not in described_compounds:
                described_compounds.append(molecule['molecule'])
            try:
                proper_molecule = wckb_metabolites[molecule['molecule']]['Name']
                #print(molecule['molecule'])
                #print(proper_molecule)
            except:
                print('abbreviation-only compound:\t', molecule['molecule'])
                proper_molecule = molecule['molecule']
                proper_molecule = re.sub('(MG_287_MONOMER.)', '', proper_molecule)
                proper_molecule = re.sub('(MG_287_MONOMER|NA)', 'unknown', proper_molecule)
            if proper_molecule not in described_compounds:
                described_compounds.append(proper_molecule)
            
print('\n\nThermodynamically described compounds: ', len(described_compounds))
export_wckb_compounds_yes_no = {}
for compound in described_compounds:
    export_wckb_compounds_yes_no[compound] = ''
    print(compound)
    
with open('2021-04-29_APF_wckb thermodynamically described compounds for ModelSEED_01.json', 'w') as output:
    json.dump(export_wckb_compounds_yes_no, output, indent = 4)'''




WCKB enzymes:		 1635
Thermodynamically described enzymes:  208
WCKB Keqs:		 197
WCKB Kinetics:		 204
WCKB gibbs:		 208
(2) H[0] + (1) PI[0] + (1) URI[0] <=> (1) R1P[0] + (1) URA[0]
(1) CO2[1] <=> (1) CO2[0]
(1) ATP[0] + (1) DCMP[0] <=> (1) ADP[0] + (1) DCDP[0]
(1) ATP[0] + (1) MG_287_MONOMER_ACP[0] + (1) OCDCA[0] <=> (1) AMP[0] + (1) MG_287_MONOMER_ocdcaACP[0] + (1) PPI[0]
(1) GN[0] + (1) PRPP[0] <=> (1) GMP[0] + (1) PPI[0]
(1) O2[1] <=> (1) O2[0]
(1) R5P[0] + (1) X5P[0] <=> (1) S7P[0] + (1) T3P1[0]
(1) E4P[0] + (1) X5P[0] <=> (1) F6P[0] + (1) T3P1[0]
(1) CTP[0] + (1) H[0] + (1) PA141[2] <=> (1) CDPDG141[2] + (1) PPI[0]
(1) ATP[0] + (1) H2O[0] + (1) MN[1] <=> (1) ADP[0] + (1) H[0] + (1) MN[0] + (1) PI[0]
(1) GTP[0] + (1) URI[0] <=> (1) GDP[0] + (1) H[0] + (1) UMP[0]
(1) AMP[0] + (1) GTP[0] <=> (1) ADP[0] + (1) GDP[0]
(1) FOR[1] <=> (1) FOR[0]
(1) ATP[0] + (1) DAMP[0] <=> (1) ADP[0] + (1) DADP[0]
(1) H[0] + (1) NADPH[0] + (1) RIBFLV[0] <=> (1) NADP[0] + (1) RBFLVRD[0]
(1) HCYS[0] + (1

"described_compounds = []\nfor enzyme in thermodynamically_described_enzymes:\n    for entry in wckb[enzyme]['stoichiometry']:\n        for molecule in entry:\n            if molecule['molecule'] not in described_compounds:\n                described_compounds.append(molecule['molecule'])\n            try:\n                proper_molecule = wckb_metabolites[molecule['molecule']]['Name']\n                #print(molecule['molecule'])\n                #print(proper_molecule)\n            except:\n                print('abbreviation-only compound:\t', molecule['molecule'])\n                proper_molecule = molecule['molecule']\n                proper_molecule = re.sub('(MG_287_MONOMER.)', '', proper_molecule)\n                proper_molecule = re.sub('(MG_287_MONOMER|NA)', 'unknown', proper_molecule)\n            if proper_molecule not in described_compounds:\n                described_compounds.append(proper_molecule)\n            \nprint('\n\nThermodynamically described compounds: ', le

The WCKB and NIST thermodynamically described datasets are combined. The exported JSON files will be used to investigate the KEGG IDs for the compounds, and the JSON files will be the subsequent basis for the TSV files that are passed throguh the ModelSEED contribution codes.  

In [201]:
import json
import re

wckb = json.load(open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_02.json'))
nist = json.load(open('2021-04-29_APF_wckb thermodynamically described compounds for ModelSEED_01.json'))
wckb_2 = json.load(open('2021-04-29_APF_wckb thermodynamically described reactions for ModelSEED_01.json'))
nist_2 = json.load(open('2021-04-29_APF_nist thermodynamically described reactions for ModelSEED_02.json'))
    
    
# combine the thermodynamically described compounds from the NIST and WCKB sources
'''export_union_compounds = {}
union_compounds = []
excluded_compounds = []
nist_compounds = []
wckb_compounds = []
for compound in nist:
    nist_compounds.append(compound)
    for compound2 in wckb:
        wckb_compounds.append(compound2)
        if compound.lower() == compound2.lower():
            if compound not in union_compounds:
                union_compounds.append(compound)
                export_union_compounds[compound] = ""

print('union compounds:\t', len(union_compounds))
for compound in union_compounds:
    print(compound)
    
complete_compounds_set = set(nist_compounds).union(set(wckb_compounds))
sorted_set_compounds = sorted(complete_compounds_set)
complete_thermodynamic_compounds = {}
print('\n\ncomplete compounds set:\t', len(complete_compounds_set))
for compound in sorted_set_compounds:
    complete_thermodynamic_compounds[compound] = ''
    print(compound)
    
    
with open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json', 'w') as output:
    json.dump(complete_thermodynamic_compounds, output, indent = 4)
    
with open('2021-04-30_APF_ModelSEED formatted compounds.tsv', 'w') as output:
    #output.write("id" + "name" + "abbreviation" + "aliases" + "formula" + "mass" + "charge" + "deltag" + "deltagerr" + "pka" + "pkb" + "inchikey" + "smiles" + "is_cofactor" + "is_core" + "is_obsolete" + "abstract_compound" + "comprised_of" + "linked_compound" + "notes" + "source" + '\n')
    output.write('\t'.join(['NAMES','ID', '\n']))
    for compound in complete_thermodynamic_compounds:
        output.write('\t'.join([compound, ]) + '\n')'''
    
    
# combine the thermodynamically described reactions from the NIST and WCKB sources
'''export_union_reactions = {}
union_reactions = []
nist_reactions = []
wckb_reactions = []
for reaction in nist_2:
    nist_reactions.append(reaction)
    for reaction2 in wckb_2:
        wckb_reactions.append(reaction2)
        if reaction.lower() == reaction2.lower():
            if reaction not in union_reactions:
                union_reactions.append(reaction)
                export_union_reactions[reaction] =''
                
print('union reactions:\t', len(union_reactions))
for reaction in union_reactions:
    print(reaction)
    
complete_reactions_set = set(nist_reactions).union(set(wckb_reactions))
sorted_set_reactions = sorted(complete_reactions_set)
complete_thermodynamic_reactions = {}
print('\n\ncomplete compounds set:\t', len(sorted_set_reactions))
for reaction in sorted_set_reactions:
    #reaction = re.sub('(\))', '\)', reaction)
    #print(reaction)
    try:
        complete_thermodynamic_reactions[reaction] = {'reaction expression':nist_2[reaction]['reaction expression'],
                                                      'yes_no':''}
    except:
        complete_thermodynamic_reactions[reaction] = {'reaction expression':wckb_2[reaction]['reaction expression'],
                                                      'yes_no':''}
    
    
with open('2021-04-30_APF_complete thermodynamically described reactions for KEGG_01.json', 'w') as output:
    json.dump(complete_thermodynamic_reactions, output, indent = 4)
    
with open('2021-04-30_APF_ModelSEED formatted reactions.tsv', 'w') as output:
    output.write('\t'.join(['NAMES','reaction', '\n']))
    for reaction in complete_thermodynamic_reactions:
        output.write('\t'.join([reaction, 
                                complete_thermodynamic_reactions[reaction]['reaction expression']]) + '\n')'''

union compounds:	 94
phosphate
uridine
uracil
carbon dioxide
ATP
dCMP
ADP
dCDP
AMP
diphosphate
guanine
GMP
D-Xylulose 5-phosphate
D-Erythrose 4-phosphate
D-Fructose 6-phosphate
CTP
GTP
GDP
UMP
Formate
dAMP
dADP
L-homocysteine
L-Methionine
5,6,7,8-Tetrahydrofolate
D-Ribulose 5-phosphate
D-alanine
pyruvate
L-tryptophan
choline
cytidine
ammonia
carbamoyl phosphate
hypoxanthine
inosine
5,10-Methylenetetrahydrofolate
adenine
glycerol
thiamine
UDP
D-Glucose 1-phosphate
D-Glucose 6-phosphate
adenosine
Acetyl phosphate
acetate
L-Arginine
L-citrulline
IMP
oxaloacetate
D-Fructose 1,6-bisphosphate
L-alanine
dUMP
dihydrofolate
dTMP
phosphoenolpyruvate
L-Glutamate
2-Oxoglutarate
L-tyrosine
guanosine
acetaldehyde
Acetyl-CoA
3-Phospho-D-glyceroyl phosphate
3-Phospho-D-glycerate
dATP
IDP
ITP
dGDP
UTP
myo-inositol
D-Fructose
D-Fructose 1-phosphate
L-phenylalanine
CDP
dTTP
CMP
L-glutamine
L-Serine
glycine
S-Adenosyl-L-homocysteine
dGMP
thymidine
Pyridoxal
D-Glucose
Dihydroxyacetone
S-Adenosyl-L-methioni

"with open('2021-04-30_APF_ModelSEED formatted reactions.tsv', 'w') as output:\n    output.write('\t'.join(['NAMES','reaction', '\n']))\n    for reaction in complete_thermodynamic_reactions:\n        output.write('\t'.join([reaction, ]) + '\n')"